In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve_triangular, norm
from scipy.optimize import minimize
import graphspme as gspme

def frobenius_norm(M):
    """M assumed sparse"""
    return np.sum(np.power(M.A, 2))

# Zero mean AR1
def rar1(n, psi):
    x = np.zeros(n)
    x[0] = np.random.normal(0, 1 / np.sqrt(1 - psi ** 2))
    w = np.random.normal(size=n - 1)
    for i in range(1, n):
        x[i] = psi * x[i - 1] + w[i - 1]
    return x

In [2]:
np.random.seed(1234)
p = 10
phi = 0.99
n = 200
x = np.vstack([rar1(p, phi) for i in range(n)])

In [3]:
data = [np.repeat(-phi, p-1), np.concatenate(([1.0], np.repeat(1.0+phi**2, p-2), [1.0])), np.repeat(-phi, p-1)]
offsets = [-1, 0, 1]
prec_pop = sparse.diags(data, offsets, shape=(p, p), format='csc')
print(prec_pop.A)

[[ 1.     -0.99    0.      0.      0.      0.      0.      0.      0.
   0.    ]
 [-0.99    1.9801 -0.99    0.      0.      0.      0.      0.      0.
   0.    ]
 [ 0.     -0.99    1.9801 -0.99    0.      0.      0.      0.      0.
   0.    ]
 [ 0.      0.     -0.99    1.9801 -0.99    0.      0.      0.      0.
   0.    ]
 [ 0.      0.      0.     -0.99    1.9801 -0.99    0.      0.      0.
   0.    ]
 [ 0.      0.      0.      0.     -0.99    1.9801 -0.99    0.      0.
   0.    ]
 [ 0.      0.      0.      0.      0.     -0.99    1.9801 -0.99    0.
   0.    ]
 [ 0.      0.      0.      0.      0.      0.     -0.99    1.9801 -0.99
   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.     -0.99    1.9801
  -0.99  ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     -0.99
   1.    ]]


In [4]:
diagonals = [[1] * p, [1] * (p - 1), [1] * (p - 1)]
Z = sparse.diags(diagonals, [0, -1, 1], format="csc")
print(Z.A)

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [5]:
prec_1 = gspme.prec_sparse(x, Z, markov_order=1, cov_shrinkage=False, symmetrization=True)
print(prec_1.A)

[[ 1.18657098 -1.1942025   0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-1.1942025   2.26320855 -1.04140628  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -1.04140628  1.9470656  -0.91155115  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.91155115  2.02436312 -1.06630997  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.06630997  2.00881192 -1.03765609
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.03765609  2.02170024
  -0.91282794  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.91282794
   1.71846669 -0.86022083  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -0.86022083  1.99275722 -1.12823449  0.        ]
 [ 0.          0.          0.          0

In [6]:
def gershgorin_spd_adjustment(prec):
    """
    Performs Gershgorin-style diagonal adjustment on the input sparse matrix `prec`
    and returns the adjusted matrix. The adjustment is performed to ensure that the matrix
    is symmetric positive definite (SPD).

    Parameters
    ----------
    prec : scipy.sparse.csc_matrix
        The input sparse matrix to adjust.

    Returns
    -------
    scipy.sparse.csc_matrix
        The SPD matrix after Gershgorin-style diagonal adjustment.
    """
    prec = prec.copy().tocsc()
    eps = 1e-1
    offdiag_abs_sum = np.abs(prec).sum(axis=1).A.ravel() - prec.diagonal()
    for i in range(prec.shape[0]):
        if offdiag_abs_sum[i] > prec[i, i]:
            prec[i, i] = offdiag_abs_sum[i] + eps
    return prec
prec_2 = gershgorin_spd_adjustment(prec_1)
print(prec_2.A)

[[ 1.2942025  -1.1942025   0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-1.1942025   2.26320855 -1.04140628  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -1.04140628  2.05295743 -0.91155115  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.91155115  2.02436312 -1.06630997  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.06630997  2.20396606 -1.03765609
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.03765609  2.02170024
  -0.91282794  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.91282794
   1.87304877 -0.86022083  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -0.86022083  1.99275722 -1.12823449  0.        ]
 [ 0.          0.          0.          0

In [7]:
perm_indices = gspme.compute_amd_ordering(prec_2)
perm_indices

array([8, 9, 7, 6, 5, 4, 3, 2, 1, 0], dtype=int32)

In [8]:
def paramIntoCholLower(theta, GraphL):
    i, j, _ = sparse.find(GraphL)
    L = csc_matrix((theta, (i, j)), shape=GraphL.shape)
    return L

def funL(thetaL, x, GraphL, perm_indices):
    L = paramIntoCholLower(thetaL, GraphL)
    return gspme.dmrfL(x, L, perm_indices)

def gradL(thetaL, x, GraphL, perm_indices):
    L = paramIntoCholLower(thetaL, GraphL)
    return gspme.dmrfL_grad(x, L, L, perm_indices).ravel()

def hessL(thetaL, x, GraphL, perm_indices):
    L = paramIntoCholLower(thetaL, GraphL)
    return gspme.dmrfL_hess(x, L, L, perm_indices)

def prec_chol_L_opt(x, L, perm_indices):
    _, _, thetaL = sparse.find(L)
    opt = minimize(
        funL,
        thetaL,
        args=(x, L, perm_indices),
        method="trust-ncg",
        #method="trust-constr",
        #method="TNC", # okay
        #method="COBYLA",
        #method="SLSQP",
        #method="Newton-CG",
        #method="cg",
        #method='BFGS',
        #method="L-BFGS-B",
        #method='trust-krylov',
        #method="dogleg",
        jac=gradL,
        hess=hessL,
        options={'disp': True,},
    )
    L = paramIntoCholLower(opt.x, L)
    return L

In [9]:
L_prev = gspme.cholesky_factor(prec_2, perm_indices)
print(L_prev.A)
#L_opt = prec_chol_L_opt(x, L)
gspme.dmrfL(x, L_prev, perm_indices)

[[ 1.03916191  0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-0.94293052  1.14245271  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -0.98755465  1.0087086   0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.85279419  1.0704162   0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.85277852  1.13774736  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -0.91202681  1.17139795
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.9102884
   1.09349812  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -0.83361017  1.16535468  0.          0.        ]
 [ 0.          0.          0.          0.        

20.203300080341005

In [10]:
L_opt = prec_chol_L_opt(x, L_prev, perm_indices)
prec_4 = gspme.chol_to_precision(L_opt, perm_indices)
print(prec_4.A)
gspme.dmrfL(x, L_opt, perm_indices)

         Current function value: 7.170507
         Iterations: 34
         Function evaluations: 35
         Gradient evaluations: 6
         Hessian evaluations: 6
[[ 0.81468149 -0.90015652  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-0.90015652  1.71772234 -0.78122622  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -0.78122622  1.79686825 -0.92925174  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.92925174  2.02850994 -1.08480081  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.08480081  2.14118291 -1.05092184
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.05092184  1.97292848
  -0.92407116  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.92407116
   1.76907406 -0.86743443  0.          0.  

/Users/berentanundstromneslunde/.pyenv/versions/3.10.0/envs/gspme-py-3.10.0/lib/python3.10/site-packages/scipy/optimize/_minimize.py:729: RuntimeWarning: A bad approximation caused failure to predict improvement.
  res = _minimize_trust_ncg(fun, x0, args, jac, hess, hessp,


7.17050726023111

In [11]:
def paramIntoPrec(theta, graph):
    i, j, _ = sparse.find(sparse.tril(graph))
    prec_ltri = sparse.csc_matrix((theta, (i, j)), shape=graph.shape)
    prec = sparse.triu(prec_ltri.transpose(), k=1) + prec_ltri
    return prec

def fun(theta, x, graph_prec, perm_indices):
    prec = paramIntoPrec(theta, graph_prec)
    return gspme.dmrf(x, prec, perm_indices)

def grad(theta, x, graph_prec, perm_indices):
    prec = paramIntoPrec(theta, graph_prec)
    return gspme.dmrf_grad(x, prec, prec).ravel()
    #return gspme.ddmrf(x, prec, perm_indices, 1.0)
    
def hess(theta, x, graph_prec, perm_indices):
    prec = paramIntoPrec(theta, graph_prec)
    return gspme.dmrf_hess(prec, sparse.tril(prec))

def prec_opt(x, prec, perm_indices):
    _, _, theta = sparse.find(sparse.tril(prec))
    opt = minimize(
        fun,
        theta,
        args=(x, prec, perm_indices),
        method="trust-ncg",
        #method="Nelder-Mead",
        #method="SLSQP",
        #method='L-BFGS-B',
        #method="TNC",
        #method='trust-constr',
        #method="trust-exact",
        #method="trust-krylov",
        #method="dogleg",
        jac=grad,
        hess=hess,
        options={'disp': False,},
    )
    prec = paramIntoPrec(opt.x, prec)
    return prec

In [12]:
#prec_5 = prec_opt(x, prec_pop, perm_indices)
prec_5 = prec_opt(x, prec_4, perm_indices)
print(prec_5.A)

[[ 1.19253219 -1.18370622  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-1.18370622  2.20922446 -1.03205711  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -1.03205711  1.96501087 -0.93630875  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.93630875  2.10668887 -1.15959284  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.15959284  2.29610706 -1.12710687
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.12710687  2.08911903
  -0.9617625   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.9617625
   1.84485933 -0.89829663  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -0.89829663  2.01703606 -1.10054785  0.        ]
 [ 0.          0.          0.          0.

In [13]:
print(frobenius_norm(prec_pop - prec_5))
print(frobenius_norm(prec_pop - prec_4))
print(gspme.dmrf(x, prec_4, perm_indices))
print(gspme.dmrf(x, prec_5, perm_indices))
print(gspme.dmrf(x, prec_pop, perm_indices))

0.473056133782124
0.44646735068134563
7.1705072602310995
6.708009323700677
6.757116042409029
